In [1]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [81]:
data = ["I love machine learning. Its awesome shirt.",
        "I love coding in python shirt",
        "I love building chatbots shirt",
        "they chat amagingly well shirt",
        "this means nothing",
        "this means everything shirt"]



tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
print(test_tagged_data)

[TaggedDocument(words=['i', 'love', 'machine', 'learning', '.', 'its', 'awesome', '.'], tags=['0']), TaggedDocument(words=['i', 'love', 'coding', 'in', 'python'], tags=['1']), TaggedDocument(words=['i', 'love', 'building', 'chatbots'], tags=['2']), TaggedDocument(words=['they', 'chat', 'amagingly', 'well'], tags=['3'])]


In [115]:
def clean_document(document):
    document = document.lower()
    default_1 = "Lightweight, Classic fit, Double-needle sleeve and bottom hem".lower()
    default_2 = "Solid colors: 100% Cotton; Heather Grey: 90% Cotton, 10% Polyester; All Other Heathers: 50% Cotton, 50% Polyester Imported Machine wash cold with like colors, dry low heat ".lower()
    document = document.replace(default_1, '').replace(default_2,'')
    
    document.replace("tee shirt","tshirt")
    document.replace("t-shirt", 'tshirt')
    document.replace(' t shirt', 'tshirt')
    document.replace("-", " ")
    
    document.replace("shirt", "")
    document.replace("tshirt", "")
    
    return document

def read_shirt_data_file(path):
    documents = []
    asins = []
    documents_dict = {}
    with open(path, 'r') as data_file:
        for line in data_file:
            line = line.split('|')
            title = line[2].split(":",1)[1]

            description = line[10].split(":",1)[1]
#             print(clean_document(title))
#             print(clean_document(description))
            document = clean_document(title) + ': ' + clean_document(description)
            
            asin = line[1].split(":")[1]
            
            documents_dict[asin] = document
            
    return documents_dict

documents_dict = read_shirt_data_file("shirts_featured1564783699")
print(documents_dict['B07MH61MWD'])

tagged_data = [TaggedDocument(words=word_tokenize(document.lower()), tags=[str(asin)]) for asin, document in documents_dict.items()]

jojo siwa dream believe achieve unicorn graphic t-shirt: officially licensed nickelodeon jojo siwa tee shirt; copyright 2019 viacom 18nkjs00009a-001 



In [116]:
max_epochs = 100
vec_size = 100
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha




model.save("d2v.model")
print("Model Saved")

iteration 0


/Users/griff/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [121]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("marvel spiderman".lower())
v1 = model.infer_vector(test_data)
print(model.docvecs.most_similar([v1]))

# to find most similar doc using tags
similar_doc = [x for x in model.docvecs.most_similar('B07V8B27DM', topn = 50) if x[1] > .5]
print(documents_dict['B07V8B27DM'])

for doc in similar_doc:
    print("{} , {} - {}".format(round(doc[1],4), doc[0], documents_dict[doc[0]]))


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['B07MH61MWD'])

[('B07B6KG8QQ', 0.6554073691368103), ('B07MNMJWS6', 0.6499558091163635), ('B07KVJWNY4', 0.6493394374847412), ('B07KVB9JYL', 0.639274537563324), ('B07H5C75TV', 0.615541398525238), ('B07DD5X33S', 0.5994100570678711), ('B07PDNWR6F', 0.5950275659561157), ('B07K986JPX', 0.5924479961395264), ('B07K97663X', 0.5920393466949463), ('B07Q39FT1C', 0.5776485204696655)]
ain't no laws when you're drinking claws t-shirt: 

0.9095 , B07V625CMZ - ain't no laws drinking claws t-shirt: ain't no laws drinking claws t-shirt 

0.8373 , B07S8JMF7K - 50th anniversary apollo 11 1969 moon landing t-shirt: 

0.8174 , B07VBN94Y7 - chris-tread shirt pratt-on don't on me tread t-shirt: 

0.8062 , B07TC6FZ36 - gm - iron maiden logo t-shirt: 

0.8005 , B07VD4MYZM - netflix stranger things 3 bitchin t-shirt: officially licensed by netflix graphic artwork: ost-0073 

0.7956 , B07TC649VX - gm - iron maiden eddie evolution t-shirt t-shirt: 

0.7844 , B07TFSQ56J - dolly parton western t-shirt: 

0.7821 , B07FN14Z2R - pound

In [63]:
print(documents_dict['B07Q96TKHF'])
print(documents_dict['B07Q7F8H8V'])
print(documents_dict['B07Q4NX2X9'])
print(documents_dict['B07MRD45W6'])
print(documents_dict['B07N3GGKZ7'])

texas tech final four t . 

Jeff Dunham Alexandria, LA Shirt. 

nipsey hussle shirt. nipsey hussle shirt nipsey hussle t shirt 

Captain Marvel Movie Goose Galaxy Portrait Graphic T Shirt. Officially Licensed Marvel Tee Shirt 18MARF00314A 

Captain Marvel Goose Full Portrait Space Graphic T Shirt. Officially Licensed Captain Marvel Tee Shirt 18MARF00741A 



In [83]:
print(model.docvecs['1'])
print(model.docvecs.most_similar('4', topn = 50))

[ 0.0244235  -0.21613473  0.18023567 -0.84176147  0.03353677  0.13153487
 -0.54832053 -0.44007263 -0.33301008  0.2449995   0.8136312   0.10832854
  0.03200603 -0.16587837  0.6732452   0.26271158 -0.7371211  -0.38190815
  0.53971565 -0.20095131]
[('5', 0.9935817122459412), ('3', 0.9903445243835449), ('0', 0.9892727136611938), ('1', 0.9883068203926086), ('2', 0.985320508480072)]


In [52]:
print(model.docvecs['1'])

[ 0.08148092 -0.18165147  0.07571121 -0.39440125 -0.01027174 -0.07501866
 -0.14441006 -0.24962953 -0.36294243  0.10393118  0.57642084 -0.1077453
 -0.11577217 -0.22908953  0.3654398   0.10156447 -0.35140935 -0.2015962
  0.35865697 -0.01273198]
